In [4]:
import os
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores.cassandra import Cassandra
import cassio

In [5]:
from langchain.vectorstores.astradb import AstraDB


In [6]:
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
api=os.getenv("GROQ_API_KEY")

In [8]:
astradb_app_token="AstraCS:CodRLeAMALWQPtZuZJOMrodz:510e65811c5303f52fe4c16568b7b4bc8d2c175244190373ecd80b0f5ed35b6f"
astradb_id="76951b4e-29b4-450f-bb6a-7a669109eebb"


In [9]:
cassio.init(token=astradb_app_token,database_id=astradb_id)

In [10]:
import bs4

In [11]:
loader=WebBaseLoader(web_path="https://lilianweng.github.io/posts/2023-06-23-agent/",
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=('post-title','post-content','post-header'))))

docs=loader.load()

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
final_docs=splitter.split_documents(docs)[:10]

In [13]:
embeddings=OllamaEmbeddings()

astra_vectordtore=Cassandra(
    embedding=embeddings,
    table_name='qa_mini_demo',
    session=None,
    keyspace=None
    
)

In [14]:
print(len(final_docs))

10


In [15]:
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

In [16]:
astra_vectordtore.add_documents(final_docs)
astradb_index=VectorStoreIndexWrapper(vectorstore=astra_vectordtore)

In [17]:
from langchain.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template('''
Answer the following question using only the provided context.
think step-by-step before generating a response.
<context>
{context}
</context>


Question:{input}''')

In [18]:
astradb_index.query("Task Decomposition",llm=ChatGroq(api_key=api))

"The Tree of Thoughts (Yao et al. 2023) is a method that extends Chain of Thought (CoT) by exploring multiple reasoning possibilities at each step, creating a tree structure of thought steps. This can be used for complex tasks, with task decomposition that can be done by a large language model (LLM), task-specific instructions, or human inputs.\n\nThe Tree of Thoughts can be used in an autonomous agent system, as shown in Fig. 1, where the first component is Planning. This includes task decomposition, which is also a component of the Chain of Thought (CoT) technique.\n\nAnother related method is Chain of Hindsight (CoH; Liu et al. 2023), which encourages the model to improve its own outputs by presenting it with a sequence of past outputs, each annotated with feedback.\n\nThe model's memory includes short-term memory, which utilizes in-context learning, and long-term memory, which allows the agent to retain and recall information over extended periods.\n\nThe agent can also learn to ca

In [19]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

retriever=astra_vectordtore.as_retriever()
doc_chain=create_stuff_documents_chain(ChatGroq(api_key=api),prompt)
ret_chain=create_retrieval_chain(retriever,doc_chain)


In [20]:
ret_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Cassandra', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.cassandra.Cassandra object at 0x0000022EFFCED750>), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context', 'input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], template='\nAnswer the following question using only the provided context.\nthink step-by-step before generating a response.\n<context>\n{context}\n</context>\n\n\nQuestion:{input}'))])
            | ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000022E88B1FC

In [21]:
response=ret_chain.invoke({'input':"Task Decomposition"})

In [23]:
response

{'input': 'Task Decomposition',
 'context': [Document(page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.\nTask decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}),
  Document(page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan 

In [24]:
print(response['answer'])

Task decomposition in the context of this text refers to the process of breaking down a complex task into smaller, manageable sub-tasks or steps. This can be done using a large language model (LLM) with simple prompting, task-specific instructions, or human inputs. An example of simple prompting for task decomposition is "Steps for XYZ.\n1." or "What are the subgoals for achieving XYZ?". Task-specific instructions and human inputs can also be used for more specialized tasks.

Chain of Thought (CoT) is a standard prompting technique that utilizes this concept of task decomposition. It instructs the model to "think step by step" to decompose hard tasks into smaller and simpler steps. This transformation of big tasks into multiple manageable tasks provides insights into the model's thinking process.

In summary, task decomposition is a crucial aspect of complex problem-solving, and it involves breaking down a problem into smaller, manageable tasks or steps to facilitate efficient problem-